## Attributes
클래스가 가지고 있는 속성에 대한 이해를 함으로써, 좀더 잘 짜여진 클래스를 만들 수 있다.  

속성에 대한 접근을 동적으로 사용자화하는 파이썬의 내장 기능들에 대해서 알아볼 것이다.  

## getter(게터)와 setter(세터)를 사용하는 이유

일반적으로 __객체의 무결성__ 을 보장하기 위해서 사용한다.  

예를 들어 man이라는 사람 클래스에 weight(몸무게)라는 필드가 있다고 해보자.  

몸무게는 음(-)의 값을 가질 수 없지만, 이 필드를 직접 수정하는 경우에는 음의 값을 임의로 지정할 수 있다.  

그렇기 때문에 setter를 통해서 이 필드를 수정함으로써, 음의값은 설정할 수 없게 구성하여 __man이라는 클래스의 weight(private으로 선언)가 0이상의 값을 가지도록__ 할 수 있다.  

getter는 단순히 해당 private 한 변수를 반환하는 함수를 의미한다.  

이 때 getter(get 메소드)를 표현할때는 __@property 데코레이터__ 를 사용한다. settet(set 메소드)를 표현할때는 __getter(get 메서드)의 이름을 `name`이라 했을때, `@name.setter`라고 표현한다.__

In [3]:
#property를 사용하지 않은 부모클래스
class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        

In [4]:
r1 = Resistor(50e3)
print(r1.ohms)

r1.ohms = 10e3
print(r1.ohms)

r1.ohms += 5e3
print(r1.ohms)

50000.0
10000.0
15000.0


In [5]:
#property를 사용한 자식클래스
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
        
    @property #getter   
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

In [6]:
r2 = VoltageResistance(1e3)
print('Before: {} amps'.format(r2.current))

r2.voltage = 10
print('After: {} amps'.format(r2.current))


Before: 0 amps
After: 0.01 amps


이를 조금더 활용하여 클래스에 전달되는 값들의 타입을 사전에 체크할 수 있다.

- case1. 클래스의 입력값의 범위를 지정하여 받고 싶은 경우
- case2. 부모클래스의 속성을 불변으로 만들고 싶은 경우

In [7]:
#case1. 클래스의 입력값의 범위를 지정하여 받고 싶은 경우
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if ohms <=0:
            raise ValueError('{}, but ohms must be > 0'.format(ohms))
        self._ohms = ohms

In [9]:
#ohms 값은 양수 값이므로, 0의 값을 넣었을 때 에러가 뜨는 것이 당연한 상황이다.
r3 = BoundedResistance(1e3)
r3.ohms = 0

ValueError: 0, but ohms must be > 0

In [10]:
#case2. 부모클래스의 속성을 불변으로 만들고 싶은 경우
class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError("Can't set attribute")
        self._ohms = ohms

In [11]:
#이미 존재하는 ohms 값을 수정하려고 할 때 에러가 뜨는 것이 당연한 상황이다.
r4 = FixedResistance(1e3)
r4.ohms = 0

AttributeError: Can't set attribute

### @property를 사용하는 다른 경우

클래스의 속성값 중 하나가 바뀜에 따라서, 다른 값도 같이 변화하는 경우에 많이 사용한다.  

예를 들어, 구멍난 양동이의 할당량과 이 할당량을 이용할 수 있는 시간에 대한 정보를 가진 클래스를 구현한다고 해보자.  

In [13]:
from datetime import datetime, timedelta

class Bucket(object):
    def __init__(self, period):
        self.period_delta = timedelta(seconds = period)
        self.reset_time = datetime.now()
        self.quota = 0
    
    def __repr__(self):
        return 'Bucket(quota = {})'.format(self.quota)
    

In [17]:
def fill(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount

In [18]:
def deduct(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        return False
    if bucket.quota - amount < 0:
        return False
    bucket.quota -= amount
    return True

In [24]:
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

Bucket(quota = 100)


In [25]:
if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')
print(bucket)

Had 99 quota
Bucket(quota = 1)


In [28]:
class Bucket(object):
    def __init__(self, period):
        self.period_delta = timedelta(seconds = period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
    
    def __repr__(self):
        return 'Bucket(max_quota = {}, quota_consumed = {})'.format(self.max_quota, self.quota_consumed)
    
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota- amount
        
        if amount == 0:
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta

In [30]:
bucket = Bucket(60)
print('Initial', bucket)

fill(bucket, 100)
print('Filled', bucket)

if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')
print('Now', bucket)

if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')
print('Now', bucket)

Initial Bucket(max_quota = 0, quota_consumed = 0)
Filled Bucket(max_quota = 100, quota_consumed = 0)
Had 99 quota
Now Bucket(max_quota = 100, quota_consumed = 99)
Not enough for 3 quota
Now Bucket(max_quota = 100, quota_consumed = 99)


`@property` 에 대해서 요약하면 다음과 같다.  
- 기존의 인스턴스 속성에 새 기능을 부여하려면 @property를 사용하자
- @property를 사용하여 점점 나은 데이터 모델로 발전시키자
- @property를 너무 많이 사용한다면 클래스와 이를 호출하는 모든 곳을 리팩토링하는 방안을 고려하자

## 재사용 가능한 @property 메서드에는 디스크립터를 사용하자

@property의 가장 큰 단점은 재사용성이다.  

다시 말해, @property로 데코레이트 하는 메서드를 같은 클래스에 속한 여러 속성에 사용하지 못한다. 또한 관련 없는 클래스에서도 재사용할 수 없다.  

예를 들어 한 과목의 시험성적을 매기는 경우에는 @property 를 사용하면 간편하지만, __여러과목의 시험성적을 매긴다__ 고 생각해보자.  

@property를 사용하는 경우, 각 과목별로 getter/setter 메서드를 각각 지정해야하기 때문에 코드가 상당히 장황해진다.  

이런 경우 __디스크립터 프로토콜(descripter protocol)__ 을 통해 수월하게 작업할 수 있다.

In [31]:
class Grade(object):
    def __get__(*args, **kwargs):
        pass

    def __set__(*args, **kwargs):
        pass

In [43]:
class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()
    

In [44]:
exam = Exam()
exam.writing_grade = 40
print(exam.writing_grade)

40


In [45]:
#case1. 클래스를 잘못 구성함으로써, 모든 exam의 인스턴스가 writing_grade 값을 공유하게 된 경우
class Grade(object):
    def __init__(self):
        self._value = 0
        
    def __get__(self, instance, instance_type):
        return self._value

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._value = value

In [46]:
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

Writing 82
Science 99


In [48]:
second_exam = Exam()
second_exam.writing_grade= 75
print('First', first_exam.writing_grade)
print('Second', second_exam.writing_grade)

First 75
Second 75


In [59]:
class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [60]:
#case2. 위의 문제를 해결하기 위해서 딕셔너리에 각 인스턴스의 상태를 저장하는 형태의 클래스를 제작
class Grade(object):
    def __init__(self):
        self._values = {}
        
    def __get__(self, instance, instance_type):
        if instance is None : return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value

In [61]:
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

second_exam = Exam()
second_exam.writing_grade= 75
print('First', first_exam.writing_grade)
print('Second', second_exam.writing_grade)

Writing 82
Science 99
First 82
Second 75


In [56]:
from weakref import WeakKeyDictionary

class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [57]:
#case3. 위의 문제를 해결하기 위해서 WeakkeyDict를 활용한 클래스를 제작


class Grade(object):
    def __init__(self):
        self._values = WeakKeyDictionary()
        
    def __get__(self, instance, instance_type):
        if instance is None : return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value

In [58]:
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

second_exam = Exam()
second_exam.writing_grade = 75
print('First', first_exam.writing_grade)
print('Second', second_exam.writing_grade)

Writing 82
Science 99
First 82
Second 75
